In [1]:
import os
import csv
import pandas as pd

In [2]:
# Set path for file
file01= "schools_complete.csv"
file02="students_complete.csv"


In [3]:
schools=pd.read_csv(file01, encoding="utf-8")
students=pd.read_csv(file02, encoding="utf-8")


In [4]:
schools

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [5]:
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [6]:
renamed_schools = schools.rename(columns={"name":"school name"})
renamed_students = students.rename(columns={"name":"student name", "school":"school name"})
renamed_schools.head()
renamed_students.head()
merged= pd.merge(renamed_schools, renamed_students, on="school name")
merged.head()

,School ID,school name,type,size,budget,Student ID,student name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [7]:
merged_district=merged.loc[merged["type"] == "District", :]
merged_district.head()

# Total budget
merged_district["Total_budget"] = merged_district["budget"].unique().sum()
total_budget=merged_district["budget"].unique().sum()

# Total number of schools 
merged_district["Total_number_of_schools"] = len(merged_district["school name"].unique())

# Total number of students
merged_district["Total_number_of_students"]= len(merged_district["student name"])
total_students=len(merged_district["student name"])

#budget per student
merged_district["budget per student"] = total_budget / total_students

# average math and reading score
merged_district["average_math_score"] =merged_district["math_score"].mean()
merged_district["average_reading_score"] =merged_district["reading_score"].mean()
merged_district=merged_district.drop(columns=["gender", "grade","School ID","Student ID"])
merged_district.head()

# percent of passing math
number_of_math_passing_student= merged_district[merged_district["math_score"] > 70].count()["school name"]
number_of_math_passing_student
merged_district["math_passing_%"]= (number_of_math_passing_student / total_students) * 100
math_passing_percent= (number_of_math_passing_student / total_students) * 100


# percent of passing reading
number_of_reading_passing_student= merged_district[merged_district["reading_score"] > 70].count()["school name"]
number_of_reading_passing_student
merged_district["reading_passing_%"]= (number_of_reading_passing_student/total_students) * 100
reading_passing_percent= (number_of_reading_passing_student/total_students) * 100


# Overall Passing Rate (Average of the above two)
overall_passing_rate= (math_passing_percent + reading_passing_percent) /2
merged_district["Overall_passing_%"] =overall_passing_rate


district_data= merged_district.loc[0,["type","Total_budget","budget per student","Total_number_of_schools","Total_number_of_students",
                                     "average_math_score","average_reading_score","math_passing_%","reading_passing_%",
                                     "Overall_passing_%"]]
                                   
# district 
district_data= pd.DataFrame(district_data) 
district_data



/Users/tarekmagdyshehatamohamed/miniconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/tarekmagdyshehatamohamed/miniconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/tarekmagdyshehatamohamed/miniconda3/envs/bioinfo/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,0
type,District
Total_budget,17347923
budget per student,643.087
Total_number_of_schools,7
Total_number_of_students,26976
average_math_score,76.987
average_reading_score,80.9625
math_passing_%,64.3053
reading_passing_%,78.3697
Overall_passing_%,71.3375


In [8]:
# Other requirments
grp = merged.groupby(['school name'])
budget=grp["budget"].unique()
num_students=grp["size"].unique()
budget_per_student= budget / num_students
school_type=grp["type"].unique()
reading_score=grp["reading_score"].mean()
math_score=grp["math_score"].mean()


#math_passing_percent by school
grp02=merged.loc[(merged["math_score"] > 70)]
grp02= grp02.groupby(["school name"])
number_of_student_passed_math =grp02["math_score"].count()
number_of_student_passed_math
math_passing_percent= (number_of_student_passed_math / num_students) *100
#math_passing_percent

#reading_passing_percent by school
grp03=merged.loc[(merged["reading_score"] > 70)]
grp03= grp03.groupby(["school name"])
number_of_student_passed_reading =grp03["reading_score"].count()
number_of_student_passed_reading
reading_passing_percent= (number_of_student_passed_reading / num_students) *100
#reading_passing_percent

# overall passing rate
overall_passing_rate = (math_passing_percent + reading_passing_percent) /2
#overall_passing_rate



In [9]:
school_data=pd.DataFrame({"Total budget" :budget,"number of student":num_students, 
                          "budget per student" : budget_per_student,
                          "type of school":school_type, "average reading score": reading_score,
                          "average math score":math_score, "reading passing %": reading_passing_percent, 
                          "math passing %": math_passing_percent,"overall passing rate":overall_passing_rate} )

school_data = school_data[["type of school","Total budget","number of student","budget per student",
                           "average reading score","average math score", "reading passing %", "math passing %","overall passing rate" ]]
school_data


,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Bailey High School,[District],[3124928],[4976],[628.0],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836]
Cabrera High School,[Charter],[1081356],[1858],[582.0],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761]
Figueroa High School,[District],[1884411],[2949],[639.0],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578]
Ford High School,[District],[1763916],[2739],[644.0],80.746258,77.102592,[77.5100401606],[65.7539247901],[71.6319824754]
Griffin High School,[Charter],[917500],[1468],[625.0],83.816757,83.351499,[93.3923705722],[89.7138964578],[91.553133515]
Hernandez High School,[District],[3022020],[4635],[652.0],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661]
Holden High School,[Charter],[248087],[427],[581.0],83.814988,83.803279,[92.7400468384],[90.6323185012],[91.6861826698]
Huang High School,[District],[1910635],[2917],[655.0],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867]
Johnson High School,[District],[3094650],[4761],[650.0],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305]


In [10]:
#schools sorted by overall passing rate
school_sorted=school_data.sort_values("overall passing rate", ascending=False)
school_sorted

,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Wilson High School,[Charter],[1319574],[2283],[578.0],83.989488,83.274201,[93.2544897065],[90.9329829172],[92.0937363119]
Pena High School,[Charter],[585858],[962],[609.0],84.044699,83.839917,[92.2037422037],[91.683991684],[91.9438669439]
Wright High School,[Charter],[1049400],[1800],[583.0],83.955000,83.682222,[93.4444444444],[90.2777777778],[91.8611111111]
Cabrera High School,[Charter],[1081356],[1858],[582.0],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761]
Holden High School,[Charter],[248087],[427],[581.0],83.814988,83.803279,[92.7400468384],[90.6323185012],[91.6861826698]
Thomas High School,[Charter],[1043130],[1635],[638.0],83.848930,83.418349,[92.9051987768],[90.2140672783],[91.5596330275]
Griffin High School,[Charter],[917500],[1468],[625.0],83.816757,83.351499,[93.3923705722],[89.7138964578],[91.553133515]
Shelton High School,[Charter],[1056600],[1761],[600.0],83.725724,83.359455,[92.617830778],[89.8921067575],[91.2549687677]
Bailey High School,[District],[3124928],[4976],[628.0],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836]


In [11]:
# Top five Performing Schools (By Passing Rate)
school_sorted=school_data.sort_values("overall passing rate", ascending=False)
school_sorted
Top_five_school= school_sorted.iloc[0:5,:]
Top_five_school

,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Wilson High School,[Charter],[1319574],[2283],[578.0],83.989488,83.274201,[93.2544897065],[90.9329829172],[92.0937363119]
Pena High School,[Charter],[585858],[962],[609.0],84.044699,83.839917,[92.2037422037],[91.683991684],[91.9438669439]
Wright High School,[Charter],[1049400],[1800],[583.0],83.955000,83.682222,[93.4444444444],[90.2777777778],[91.8611111111]
Cabrera High School,[Charter],[1081356],[1858],[582.0],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761]
Holden High School,[Charter],[248087],[427],[581.0],83.814988,83.803279,[92.7400468384],[90.6323185012],[91.6861826698]


In [12]:
#Bottom five performing schools sorted by overall passing rate
school_sorted=school_data.sort_values("overall passing rate")
school_sorted
Bottom_five_school= school_sorted.iloc[0:5,:]
Bottom_five_school

,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Rodriguez High School,[District],[2547363],[3999],[637.0],80.744686,76.842711,[77.744436109],[64.0660165041],[70.9052263066]
Huang High School,[District],[1910635],[2917],[655.0],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867]
Johnson High School,[District],[3094650],[4761],[650.0],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305]
Figueroa High School,[District],[1884411],[2949],[639.0],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578]
Hernandez High School,[District],[3022020],[4635],[652.0],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661]


In [13]:
school_sorted_budget=school_data.sort_values("budget per student", ascending=False)
school_sorted_budget



,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Huang High School,[District],[1910635],[2917],[655.0],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867]
Hernandez High School,[District],[3022020],[4635],[652.0],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661]
Johnson High School,[District],[3094650],[4761],[650.0],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305]
Ford High School,[District],[1763916],[2739],[644.0],80.746258,77.102592,[77.5100401606],[65.7539247901],[71.6319824754]
Figueroa High School,[District],[1884411],[2949],[639.0],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578]
Thomas High School,[Charter],[1043130],[1635],[638.0],83.848930,83.418349,[92.9051987768],[90.2140672783],[91.5596330275]
Rodriguez High School,[District],[2547363],[3999],[637.0],80.744686,76.842711,[77.744436109],[64.0660165041],[70.9052263066]
Bailey High School,[District],[3124928],[4976],[628.0],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836]
Griffin High School,[Charter],[917500],[1468],[625.0],83.816757,83.351499,[93.3923705722],[89.7138964578],[91.553133515]


In [14]:
#school scores by budget per student
bins = [0, 600, 630, 700]
group_names = ['low', 'moderate', 'high']
school_sorted_budget["budget per student group"] =pd.cut(school_sorted_budget["budget per student"], bins, labels = group_names)
school_sorted_budget_final = school_sorted_budget.drop(columns=["type of school", "number of student", "Total budget"])
school_sorted_budget_final


,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate,budget per student group
school name,,,,,,,
Huang High School,[655.0],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867],high
Hernandez High School,[652.0],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661],high
Johnson High School,[650.0],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305],high
Ford High School,[644.0],80.746258,77.102592,[77.5100401606],[65.7539247901],[71.6319824754],high
Figueroa High School,[639.0],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578],high
Thomas High School,[638.0],83.848930,83.418349,[92.9051987768],[90.2140672783],[91.5596330275],high
Rodriguez High School,[637.0],80.744686,76.842711,[77.744436109],[64.0660165041],[70.9052263066],high
Bailey High School,[628.0],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836],moderate
Griffin High School,[625.0],83.816757,83.351499,[93.3923705722],[89.7138964578],[91.553133515],moderate


In [15]:
school_sorted_size=school_data.sort_values("number of student", ascending=False)
school_sorted_size


,type of school,Total budget,number of student,budget per student,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,,,,
Bailey High School,[District],[3124928],[4976],[628.0],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836]
Johnson High School,[District],[3094650],[4761],[650.0],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305]
Hernandez High School,[District],[3022020],[4635],[652.0],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661]
Rodriguez High School,[District],[2547363],[3999],[637.0],80.744686,76.842711,[77.744436109],[64.0660165041],[70.9052263066]
Figueroa High School,[District],[1884411],[2949],[639.0],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578]
Huang High School,[District],[1910635],[2917],[655.0],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867]
Ford High School,[District],[1763916],[2739],[644.0],80.746258,77.102592,[77.5100401606],[65.7539247901],[71.6319824754]
Wilson High School,[Charter],[1319574],[2283],[578.0],83.989488,83.274201,[93.2544897065],[90.9329829172],[92.0937363119]
Cabrera High School,[Charter],[1081356],[1858],[582.0],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761]


In [16]:
#school scores by  size (number of students)

bins = [0, 1000, 2000, 5000]
group_names = ['small', 'medium', 'large']
school_sorted_size["size group"] =pd.cut(school_sorted_size["number of student"], bins, labels = group_names)
school_sorted_size
school_sorted_size_final = school_sorted_size.drop(columns=["type of school", "budget per student", "Total budget"])
school_sorted_size_final

,number of student,average reading score,average math score,reading passing %,math passing %,overall passing rate,size group
school name,,,,,,,
Bailey High School,[4976],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836],large
Johnson High School,[4761],80.966394,77.072464,[78.281873556],[63.8521319051],[71.0670027305],large
Hernandez High School,[4635],80.934412,77.289752,[78.1877022654],[64.7464940669],[71.4670981661],large
Rodriguez High School,[3999],80.744686,76.842711,[77.744436109],[64.0660165041],[70.9052263066],large
Figueroa High School,[2949],81.158020,76.711767,[78.4333672431],[63.7504238725],[71.0918955578],large
Huang High School,[2917],81.182722,76.629414,[78.8138498457],[63.3184778882],[71.066163867],large
Ford High School,[2739],80.746258,77.102592,[77.5100401606],[65.7539247901],[71.6319824754],large
Wilson High School,[2283],83.989488,83.274201,[93.2544897065],[90.9329829172],[92.0937363119],large
Cabrera High School,[1858],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761],medium


In [17]:
#school scores by  type (district_vs_charter)
district_vs_charter= school_data.drop(columns=["number of student", "budget per student", "Total budget"])
district_vs_charter.sort_values("type of school")

,type of school,average reading score,average math score,reading passing %,math passing %,overall passing rate
school name,,,,,,
Cabrera High School,[Charter],83.975780,83.061895,[93.8643702906],[89.5586652314],[91.711517761]
Griffin High School,[Charter],83.816757,83.351499,[93.3923705722],[89.7138964578],[91.553133515]
Holden High School,[Charter],83.814988,83.803279,[92.7400468384],[90.6323185012],[91.6861826698]
Pena High School,[Charter],84.044699,83.839917,[92.2037422037],[91.683991684],[91.9438669439]
Shelton High School,[Charter],83.725724,83.359455,[92.617830778],[89.8921067575],[91.2549687677]
Thomas High School,[Charter],83.848930,83.418349,[92.9051987768],[90.2140672783],[91.5596330275]
Wilson High School,[Charter],83.989488,83.274201,[93.2544897065],[90.9329829172],[92.0937363119]
Wright High School,[Charter],83.955000,83.682222,[93.4444444444],[90.2777777778],[91.8611111111]
Bailey High School,[District],81.033963,77.048432,[79.3006430868],[64.6302250804],[71.9654340836]
